```python
# Copyright (c) 2023 Microsoft Corporation
# Copyright (c) 2024 Bytedance Ltd. and/or its affiliates
# SPDX-License-Identifier: Apache-2.0
#
# This file has been modified by Xinyou Wang on Jul 21, 2024
#
# Original file was released under MIT, with the full license text
# available at https://github.com/microsoft/evodiff/blob/main/LICENSE
#
# This modified file is released under the same license.
```

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import os
import MDAnalysis as mda
from MDAnalysis.analysis import rms
from ast import literal_eval
import subprocess

In [3]:
start_idx_dict = {
    '1prw': [15, 51],
    '1bcf': [17, 46, 90, 122],
    '5tpn': [108],
    '3ixt': [0],
    '4jhw': [37, 144],
    '4zyp': [357],
    '5wn9': [1],
    '5ius': [34, 88],
    '5yui': [89, 114, 194],
    '6vw1': [5, 45],
    '1qjg': [13, 37, 98],
    '1ycr': [2],
    '2kl8': [0, 27],
    '7mrx': [25],
    '5trv': [45],
    '6e6r': [22],
    '6exz': [25],
}

end_idx_dict = {
    '1prw': [34, 70],
    '1bcf': [24, 53, 98, 129],
    '5tpn': [126],
    '3ixt': [23],
    '4jhw': [43, 159],
    '4zyp': [371],
    '5wn9': [20],
    '5ius': [53, 109],
    '5yui': [93, 116, 196],
    '6vw1': [23, 63],
    '1qjg': [13, 37, 98],
    '1ycr': [10],
    '2kl8': [6, 78],
    '7mrx': [46],
    '5trv': [69],
    '6e6r': [34],
    '6exz': [39],
}

def calc_rmsd_tmscore(pdb_name, reference_PDB, scaffold_pdb_path=None, scaffold_info_path=None, ref_motif_starts=[30], ref_motif_ends=[44],
              output_path=None):
    "Calculate RMSD between reference structure and generated structure over the defined motif regions"

    motif_df = pd.read_csv(os.path.join(scaffold_info_path, f'{pdb_name}.csv'), index_col=0) #, nrows=num_structures)
    results = []
    for pdb in os.listdir(os.path.join(scaffold_pdb_path, f'{pdb_name}')): # This needs to be in numerical order to match new_starts file
        if not pdb.endswith('.pdb'):
            continue
        ref = mda.Universe(reference_PDB)
        predict_PDB = os.path.join(os.path.join(scaffold_pdb_path, f'{pdb_name}'), pdb)
        u = mda.Universe(predict_PDB)

        ref_selection = 'name CA and resnum '
        u_selection = 'name CA and resnum '
        i = int(pdb.split('_')[1])
        new_motif_starts = literal_eval(motif_df['start_idxs'].iloc[i])
        new_motif_ends = literal_eval(motif_df['end_idxs'].iloc[i])

        for j in range(len(ref_motif_starts)):
            ref_selection += str(ref_motif_starts[j]) + ':' + str(ref_motif_ends[j]) + ' ' 
            u_selection += str(new_motif_starts[j]+1) + ':' + str(new_motif_ends[j]+1) + ' '
        # print("U SELECTION", u_selection)
        # print("SEQUENCE", i)
        # print("ref", ref.select_atoms(ref_selection).resnames)
        # print("gen", u.select_atoms(u_selection).resnames)
        # This asserts that the motif sequences are the same - if you get this error something about your indices are incorrect - check chain/numbering
        assert len(ref.select_atoms(ref_selection).resnames) == len(u.select_atoms(u_selection).resnames), "Motif \
                                                                    lengths do not match, check PDB preprocessing\
                                                                    for extra residues"

        assert (ref.select_atoms(ref_selection).resnames == u.select_atoms(u_selection).resnames).all(), "Resnames for\
                                                                    motifRMSD do not match, check indexing"
        rmsd = rms.rmsd(u.select_atoms(u_selection).positions,
                        # coordinates to align
                        ref.select_atoms(ref_selection).positions,
                        # reference coordinates
                        center=True,  # subtract the center of geometry
                        superposition=True)  # superimpose coordinates
        
        temp_file = open(os.path.join(output_path, 'temp_tmscores.txt'), 'w')

        subprocess.call(['./TMscore', reference_PDB,
                            predict_PDB,  '-seq'],
                        stdout=temp_file)
        with open(os.path.join(output_path, 'temp_tmscores.txt'), 'r') as f:
            for line in f:
                if len(line.split())>1 and "TM-score" == line.split()[0]:
                    tm_score = line.split()[2]
                    break
        
        plddt = float(predict_PDB.split('_')[-1][:-4])
        results.append((pdb_name, i, rmsd, plddt, tm_score))
    return results
    

In [ ]:
scaffold_dir = "../generation-results/dplm_650m_scaffold"
motif_pdb_dir = '../data-bin/scaffolding-pdbs'
output_dir = os.path.join(scaffold_dir, 'scaffold_results')
os.makedirs(output_dir, exist_ok=True)

results = []
for pdb in start_idx_dict.keys():
    print(pdb)
    ref_motif_starts = start_idx_dict[pdb]
    ref_motif_ends = end_idx_dict[pdb]
    reference_PDB = os.path.join('../data-bin/scaffolding-pdbs', pdb+'_reference.pdb')
    with open(reference_PDB) as f:
        line = f.readline()
        ref_basenum = int(line.split()[5])
    ref_motif_starts = [num + ref_basenum for num in ref_motif_starts]
    ref_motif_ends = [num + ref_basenum for num in ref_motif_ends]
    results += (
        calc_rmsd_tmscore(
                pdb_name=pdb,
                reference_PDB=reference_PDB,
                scaffold_pdb_path=f'{scaffold_dir}/scaffold_fasta/esmfold_pdb',
                scaffold_info_path=f'{scaffold_dir}/scaffold_info',
                ref_motif_starts=ref_motif_starts,
                ref_motif_ends=ref_motif_ends,
                output_path=output_dir,
            )
        )
    
results = pd.DataFrame(results, columns=['pdb_name', 'index', 'rmsd', 'plddt', 'tmscore'])
results.to_csv(os.path.join(output_dir, 'rmsd_tmscore.csv'), index=False)

    

In [5]:
rmsd_tmscore = pd.read_csv(os.path.join(output_dir, 'rmsd_tmscore.csv'))
rmsd_tmscore


,pdb_name,index,rmsd,plddt,tmscore
0,1prw,0,0.618942,70.092964,0.6155
1,1prw,1,0.733485,75.488167,0.4946
2,1prw,2,0.618464,75.638611,0.4677
3,1prw,3,0.713177,78.954086,0.5017
4,1prw,4,0.837493,70.736710,0.7587
...,...,...,...,...,...
80,6exz,0,4.712714,76.886765,0.1707
81,6exz,1,3.366002,35.043579,0.1983
82,6exz,2,4.577574,85.407104,0.2019
83,6exz,3,4.661258,84.473694,0.1608


In [37]:
def cal_success_scaffold(pdb):
    total = len(pdb)
    pdb['total'] = total
    pdb = pdb[(pdb['rmsd'] < 1.0) & (pdb['plddt'] > 70)]
    return pdb
success_scaffold = rmsd_tmscore.groupby('pdb_name', as_index=False).apply(cal_success_scaffold)
# success_scaffold.to_csv(os.path.join(output_dir, 'success_scaffold.csv'), index=False)
success_scaffold_count = success_scaffold.groupby('pdb_name').size()
success_scaffold_count = success_scaffold_count.reset_index(name='success_count')

all_pdb = list(rmsd_tmscore['pdb_name'].unique())
success_pdb = list(success_scaffold_count['pdb_name'])
failed_pdb = list(set(all_pdb)-set(success_pdb))
failed_scaffold_count = {
    'pdb_name': failed_pdb,
    'success_count': [0] * len(failed_pdb),
}
results = pd.concat([success_scaffold_count, pd.DataFrame(failed_scaffold_count)]).sort_values('pdb_name')
results.to_csv(os.path.join(output_dir, 'result.csv'))


/tmp/ipykernel_29139/408737312.py:6: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  success_scaffold = rmsd_tmscore.groupby('pdb_name', as_index=False).apply(cal_success_scaffold)
